# Accessing  HEASARC tables through the TAP with ADQL

We have used the __[Table Access Protocol](http://www.ivoa.net/documents/TAP/)__ (TAP) protocol in several other notebooks for basic queries.  Here, we expand on its usage and that of the __[Astronomical Data Query Language](http://www.ivoa.net/documents/latest/ADQL.html)__ (ADQL) that it uses.  

* [1. Basic](#basic) Table Access Protocol queries
* [2. Cross-correlating](#cc) our own catalog with a HEASARC catalog
* [3. Combining](#combo) data from multiple catalogs and cross-correlating


In [2]:
import numpy
## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

## For simple astropy tables
import astropy, io, requests

## For handling ordinary astropy Tables
from astropy.table import Table

## For handling VO table type objects
from astropy.io import votable as apvot

# Use the astroquery TapPlus library.
from astroquery.utils.tap.core import TapPlus

## Use NAVO utility for Registry and Cone searches
import sys
sys.path.append("../aas_workshop_2018_summer/navo_utils")
from navo_utils.registry import Registry
from navo_utils.cone import Cone
from navo_utils.tap import Tap


<a id="basic"></a>

# 1. Basic Table Access Protocol queries

A TAP query is the most powerful way to search a catalog. A Simple Cone Search only allows you to ask for a position and radius, but TAP allows you to do much more.    

Supposed you already know that you want to query the "zcat" at the HEASARC. (More on discovering catalogs to query below.)  We use the registry to find the corresponding cone service and then submit our cone search.  

In [23]:
import astropy.coordinates as coord
coord=coord.SkyCoord.from_name("m51")
print(coord)
services=Registry.query(service_type='cone',source='heasarc%zcat')
services

<SkyCoord (ICRS): (ra, dec) in deg
    (202.469575, 47.1952583)>


waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
str23,str9,str28,str24,str77,str57,str17,str10
optical,ABELLZCAT,ivo://nasa.heasarc/abellzcat,No Description Available,https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=abellzcat&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/abellzcat.html,NASA/GSFC HEASARC,conesearch
gamma-ray#optical#x-ray,ROMABZCAT,ivo://nasa.heasarc/romabzcat,No Description Available,https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=romabzcat&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/romabzcat.html,NASA/GSFC HEASARC,conesearch
optical,CFAZ,ivo://nasa.heasarc/zcat,No Description Available,https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=zcat&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/zcat.html,NASA/GSFC HEASARC,conesearch


In [24]:
## Different tries seem to come back in different order!  Want CFAZ.  
table=Cone.query(service=services[2],coords=coord,radius=1)
table[0]

name,ra,dec,bmag,radial_velocity,radial_velocity_error,redshift,class,Search_Offset
,deg,deg,,km / s,km / s,,,
str10,float64,float64,float32,int32,int32,float64,int32,float64
13267+4631,202.20650,46.25850,15.10,0,0,--,9999,57.2374
N5173,202.10529,46.59082,14.12,2467,26,--,6300,39.2217
N5198,202.54745,46.66996,13.30,2569,25,--,6300,31.6790
N5169,202.04237,46.67154,14.70,2482,40,--,6200,35.9691
I4263,202.13883,46.92671,15.40,2663,25,--,6200,21.0327
N5194,202.46823,47.19815,9.03,474,23,--,6200,0.1819
N5195,202.49491,47.26792,10.94,558,23,--,6200,4.4802
1325+4754,201.87771,47.64124,--,18107,179,--,9999,35.9641


With the TAP, you can refine the search based on any other attribute in the given catalog.  

Now, we saw above with the results of the Registry query for cone services that the HEASARC lists every catalog as a separate cone service.  So we can find it with a search for source='heasarc%zcat', where the "%" is a wildcard.  But for TAP services, HEASARC lists in the Registry one TAP service that can access many catalogs:

In [16]:
tap_services=Registry.query(service_type='table',source='heasarc')
url=tap_services[0]['access_url']
print(url)

https://heasarc.gsfc.nasa.gov/xamin/vo/tap


In [46]:
tap_services=Registry.query(service_type='table')
tap_services[20]

waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
str16,str16,str45,str3760,str58,str155,str93,str11
x-ray,CDA,ivo://cxc.harvard.edu/cda,"The Chandra X-ray Observatory is the U.S. follow-on to the Einstein Observatory. Chandra was formerly known as AXAF, the Advanced X-ray Astrophysics Facility, but renamed by NASA in December, 1998. Originally three instruments and a high-resolution mirror carried in one spacecraft, the project was reworked in 1992 and 1993. The Chandra spacecraft carries a high resolution mirror, two imaging detectors, and two sets of transmission gratings. Important Chandra features are: an order of magnitude improvement in spatial resolution, good sensitivity from 0.1 to 10 keV, and the capability for high spectral resolution observations over most of this range.",http://cda.harvard.edu/cxctap,http://cxc.harvard.edu/cda/,Chandra X-ray Observatory,tableaccess


How do we know which we want if we did not know it was called "public.zcat"?  You can also use TapPlus to get the names of all of the catalogs a given service provides.  This could be done in a loop over *all* the Registry results for table services if you don't know which service provides the one you want.  

In [43]:
cxc = TapPlus(url=tap_services[20]['access_url'])
tables = cxc.load_tables()
for table in (tables):
    print(table.get_qualified_name())


Created TAP+ (v1.0.1) - Connection:
	Host: cda.harvard.edu
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Retrieving tables...
Parsing tables...
Done.
TAP_SCHEMA.TAP_SCHEMA.schemas
TAP_SCHEMA.TAP_SCHEMA.tables
TAP_SCHEMA.TAP_SCHEMA.columns
TAP_SCHEMA.TAP_SCHEMA.keys
TAP_SCHEMA.TAP_SCHEMA.key_columns
ivoa.ivoa.ObsCore
cxc.cxc.proposal
cxc.cxc.observation
cxc.cxc.image
cxc.cxc.cone


In [45]:
cxc_table = cxc.load_table('cxc.cxc.image')
for column in (cxc_table.get_columns()):
    print(column.get_name())


Retrieving table 'cxc.cxc.image'
Parsing table 'cxc.cxc.image'...
Done.
schema_name
description
utype
id


Then after you've selected the *public.zcat* table, you want to know what columns you can query:  

In [49]:
## Use the Registry to find all TAP services and select one, which CXC
tap_services=Registry.query(service_type='table')
print(tap_services[20]['access_url'])

## Then get all the tables served by this service:
cxc = TapPlus(url=tap_services[20]['access_url'])
tables = cxc.load_tables()
for table in (tables):
    print(table.get_qualified_name())

## Then try to find the columns
cxc_table = cxc.load_table('cxc.cxc.image')
for column in (cxc_table.get_columns()):
    print(column.get_name())


http://cda.harvard.edu/cxctap
Created TAP+ (v1.0.1) - Connection:
	Host: cda.harvard.edu
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Retrieving tables...
Parsing tables...
Done.
TAP_SCHEMA.TAP_SCHEMA.schemas
TAP_SCHEMA.TAP_SCHEMA.tables
TAP_SCHEMA.TAP_SCHEMA.columns
TAP_SCHEMA.TAP_SCHEMA.keys
TAP_SCHEMA.TAP_SCHEMA.key_columns
ivoa.ivoa.ObsCore
cxc.cxc.proposal
cxc.cxc.observation
cxc.cxc.image
cxc.cxc.cone
Retrieving table 'cxc.cxc.image'
Parsing table 'cxc.cxc.image'...
Done.
schema_name
description
utype
id


## Why does this work for this Gaia table but not for HEASARC's zcat or CXC stuff?

In [26]:
gaia = TapPlus(url="http://gea.esac.esa.int/tap-server/tap")
gaiadr1_table = gaia.load_table('gaiadr1.gaia_source')
for column in (gaiadr1_table.get_columns()):
    print(column.get_name())


Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Retrieving table 'gaiadr1.gaia_source'
Parsing table 'gaiadr1.gaia_source'...
Done.
solution_id
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_good_obs_ac
astrometric_n_bad_obs_al
astrometric_n_bad_obs_ac
astrometric_delta_q
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_primary_flag
astrometric_relegation_factor
astrometric_weight_al
astrometric_weight_ac
astrometric_priors_used
matched_observations
duplicated_source
scan_direction_strength_k1
scan_direction_strength_k2
scan_direction_strength_k3
scan_direction_strength_k4
scan_direction_mean_k1
scan_direction_

If you look at this url, you'll see the raw XML that it's using.  

http://gea.esac.esa.int/tap-server/tap/tables 

For this Gaia example, it's parsing what you see correctly and finding the gaiadr1.gaia_source table somewhere down in the middle.

But if you look at the HEASARC XML, you see

view-source:https://heasarc.gsfc.nasa.gov/xamin/vo/tap/tables

and the very first table there is the TAP_SCHEMA.columns, and TapPlus is listing its columns instead of finding the public.zcat table which is further down.  So TapPlus and our TAP service tables endpoint are not playing correctly.  This is apparently the same for Chandra as well.

In [27]:
heasarc_tap = TapPlus(url=url)
zcat_table=heasarc_tap.load_table('public.zcat')
for column in (zcat_table.get_columns()):
    print(column.get_name())

Created TAP+ (v1.0.1) - Connection:
	Host: heasarc.gsfc.nasa.gov
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Retrieving table 'public.zcat'
Parsing table 'public.zcat'...
Done.
datatype
description
column_name
size
indexed
std
utype
ucd
unit
principal
table_name


In [28]:
heasarc_tap = TapPlus(url=url)
table=heasarc_tap.load_table('public.ascao')
for column in (table.get_columns()):
    print(column.get_name())

Created TAP+ (v1.0.1) - Connection:
	Host: heasarc.gsfc.nasa.gov
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Retrieving table 'public.ascao'
Parsing table 'public.ascao'...
Done.
datatype
description
column_name
size
indexed
std
utype
ucd
unit
principal
table_name


##  If you ever get the above working, then here's the rest about ADQL with examples

So lets reproduce the cone search but using TAP and a query expressed in ADQL.  

The basics of ADQL:

* *SELECT &#42; FROM my.interesting.catalog as cat* 

says you want all ("&#42;") columns from the catalog called "my.interesting.catalog", which you will refer to below by the more compact name of "cat".  

Instead of returning all columns, you can 

* *SELECT cat.RA, cat.DEC, cat.bmag from catalog as cat...* 

to only return the columns you're interested in. To use multiple catalogs, your query could start, e.g.,

* *SELECT c1.RA,c1.DEC,c2.BMAG FROM catalog1 as c1 natural join catalog2 as c2* 

says that you want to query two catalogs zipped together the "natural" way, i.e., by looking for a common column.

To select only some rows of the catalog based on the value in a column, you can add:  

* *WHERE cat.bmag < 14* 

says that you want to retrieve only those entries in the catalog whose bmag column has a value less than 14.

You can also append 

* *ORDER by cat.bmag* 

to return the result sorted ascending by one of the columns, adding *DESC* to the end for descending. 

A few special functions in the ADQL allow you to query regions:

* *WHERE contains( point('ICRS', cat.ra, cat.dec), circle('ICRS', 210.5, -6.5, 0.5))=1*

is how you would ask for any catalog entries whose RA,DEC lie within a circular region defined by RA,DEC 210.5,-6.5 and a radius of 0.5 (all in degrees).  The 'ICRS' specifies the coordinate system.  

See the ADQL documentation for more.

An ADQL query might look like this (we will discuss how to define the table and column names below):

In [20]:
##  Recall that in Python3, f"this is the value of my variable: {variable}" 
##  will print the value of the variable enclosed by the {}.  So this asks for 
##  rows of public.zcat where the row's ra and dec are within radius 1deg of the
##  given RA and DEC we got above for M51, and where the bmag column is less than 14.  
query=f"""SELECT ra, dec, Radial_Velocity FROM public.zcat as cat where 
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',{coord.ra.deg},{coord.dec.deg},{1.0}))=1 and
    cat.bmag < 14
    order by cat.radial_velocity_error
    """

In [21]:
heasarc_tap = TapPlus(url=url,verbose=True)
heasarc_tap.launch_job(query).get_results()

Created TAP+ (v1.0.1) - Connection:
	Host: heasarc.gsfc.nasa.gov
	Use HTTPS: True
	Port: 443
	SSL Port: 443


ra,dec,radial_velocity
float64,float64,int32
202.46823227,47.19814872,474
202.49490508,47.2679204,558
202.54744757,46.66995898,2569


### TO BE REMOVED?  So what do we need our own Tap for?

In [13]:
query=f"SELECT ra, dec, Radial_Velocity FROM public.zcat as cat where contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',{coord.ra.deg},{coord.dec.deg},1.0))=1 and cat.bmag < 14 order by cat.radial_velocity_error"
service=tap_services[0]
table=Tap.query(service,query)
table

ra,dec,radial_velocity
float64,float64,int32
202.46823227,47.19814872,474
202.49490508,47.2679204,558
202.54744757,46.66995898,2569


If you aren't sure what columns are available, get all attributes of one row and take a look:

In [22]:
url=str(tap_services['access_url'][0])
print(type(str(url)))
table=Tap.query(url,"SELECT top 1 * FROM zcat")
table

TypeError: string indices must be integers

(See the __[information on the zcat](https://heasarc.gsfc.nasa.gov/W3Browse/galaxy-catalog/zcat.html)__ for column information. We will use the 'radial_velocity' column rather than the 'redshift' column.)

<a id="cc"></a>
# 2. TAP:  Using the TAP to cross-correlate our objects with a catalog

Now to search all of our sources in one go, we need to upload our own table and do a 'cross-correlation' with the *zcat* table. For more on creating VOTable objects, see that notebook.  Here, we just read one in:  

This is how we'd like it to work, but for the moment, this is broken.

In [8]:
#query="""
#    SELECT cat.ra, cat.dec, Radial_Velocity 
#    FROM zcat cat, tap_upload.mysources mt 
#    WHERE
#    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,0.1))=1
#    and Radial_Velocity > 0
#    ORDER by cat.ra"""
#url='https://heasarc.gsfc.nasa.gov/xamin_test/vo/tap'
#table=Tap.query(url,query,upload_file="../my_sources.xml",upload_name='mysources')
#table.meta

### This can be done with requests:

(These take a while, i.e. half a minute.)

In [9]:
files={'uplt':open('../my_sources.xml', 'rb')}

cc_params={
    'lang': 'ADQL', 
    'request': 'doQuery',
    'upload':'mysources,param:uplt'
    }

cc_params["query"]=query
r = requests.post(tap_services[0]['access_url']+'/sync',data=cc_params,stream=True,files=files)
#r.text
mytable=Table.read(io.BytesIO(r.content))
mytable

ra,dec,radial_velocity
float64,float64,int32
202.46823227,47.19814872,474
202.49490508,47.2679204,558
202.54744757,46.66995898,2569


###  Or with TapPlus

In [10]:
from astroquery.utils.tap.core import TapPlus
xamin=TapPlus(url=tap_services[0]['access_url'])
job=xamin.launch_job(query=query, upload_resource='../my_sources.xml', upload_table_name="mysources", verbose=True)
result = job.get_results()
result.pprint()

Created TAP+ (v1.0.1) - Connection:
	Host: heasarc.gsfc.nasa.gov
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Launched query: 'SELECT  TOP 2000 ra, dec, Radial_Velocity FROM zcat as cat where contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',202.469575,47.1952583,1.0))=1 and cat.bmag < 14 order by cat.radial_velocity_error'
200 OK
[('Date', 'Fri, 01 Jun 2018 19:30:02 GMT'), ('Query-Defer', '261'), ('Content-Type', 'text/xml'), ('Transfer-Encoding', 'chunked'), ('Strict-Transport-Security', 'max-age=31536000; includeSubDomains')]
Retrieving sync. results...
Query finished.
     ra          dec     radial_velocity
------------ ----------- ---------------
202.46823227 47.19814872             474
202.49490508  47.2679204             558
202.54744757 46.66995898            2569


<a id="combo"></a>

# 3.  Combining data from different catalogs and cross-correlating
Now we'd like to take the redshift information (above, as a radial velocity) and determine a search radius to use for each galaxy based on its distance, so that we are are searching within a given physical distance. 

In [11]:
## The radial_velocity is in km/s, and this is just c*z, so
c=3.0e5 # km/s
redshifts=mytable['radial_velocity'].filled(0.)/c  # Filling masked values with zero
mytable['redshift']=redshifts
from astropy import units
physdist=0.05*units.Mpc # 50 kpc physical distance

## This needs scipy.  
from astropy.cosmology import Planck15
angDdist=Planck15.angular_diameter_distance(mytable['redshift'])
## angDdist is returned from the astropy.cosmology module as a Quantity object, 
##  i.e. a value and a unit.  Arctan is smart enough not to operate on quantities
##  that aren't unitless.  So angDdist.value to just get the value.
angDrad=numpy.arctan(physdist/angDdist)
angDdeg=angDrad*units.deg/units.rad
mytable['angDdeg']=angDdeg
mytable

ra,dec,radial_velocity,redshift,angDdeg
,,,,deg
float64,float64,int32,float64,float64
202.46823227,47.19814872,474,0.00158,0.007164315980361076
202.49490508,47.2679204,558,0.00186,0.006087942574467991
202.54744757,46.66995898,2569,0.008563333333333333,0.0013332722231319696


This time, rather than write the table to disk, we'll keep it in memory and give requests a "file-like" object using io.BytesIO():

In [12]:
## In memory only, use an IO stream. 
vot_obj=io.BytesIO()
print(mytable.columns)
apvot.writeto(apvot.from_table(mytable),vot_obj)
## (Reset the "file-like" object to the beginning.)
vot_obj.seek(0)
## 'uplt' is what we'll call it (for 'upload table') 
##   in the requests parameters below, or what you will:
files={'uplt':vot_obj}


<TableColumns names=('ra','dec','radial_velocity','redshift','angDdeg')>


This takes half a minute:

In [13]:
cc_params={
    'lang': 'ADQL', 
    'request': 'doQuery',
    'upload':'mytable,param:uplt'
    }
## This is your ADQL query, where "mytable" here has to 
##  match what you specified in the upload parameter above
cc_params["query"]="""
    SELECT cat.ra, cat.dec, cat.Radial_Velocity 
    FROM zcat cat, tap_upload.mytable mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,mt.angDdeg))=1
    and cat.Radial_Velocity > 0
    ORDER by cat.ra"""
## The name you give here (tab1) matches what's in the cc_params
r = requests.post('https://heasarc.gsfc.nasa.gov/xamin/vo/tap/sync',data=cc_params,stream=True,files=files)
mytable=Table.read(io.BytesIO(r.content))
mytable

ra,dec,radial_velocity
float64,float64,int32
202.46823227,47.19814872,474
202.49490508,47.2679204,558
202.54744757,46.66995898,2569
